stuff to implement
* Get CCG token
* /ask endpoint with hub

In [13]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_HUBS_ID=os.getenv('HUBS_ID')

print("BOX_HUBS_ID:", BOX_HUBS_ID)

BOX_HUBS_ID: 402359053


In [14]:
from box_sdk_gen import BoxClient, CCGConfig, BoxCCGAuth

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    user_id=BOX_USER_ID,
)

ccg_auth = BoxCCGAuth(ccg_config)

client = BoxClient(ccg_auth)

print(f"{client.users.get_user_me()}")

<class 'box_sdk_gen.schemas.user_full.UserFull'> {'id': '19498290761', 'type': 'user', 'name': 'Scott Hurrey', 'login': 'shurrey+eplusadmin@boxdemo.com', 'created_at': '2022-05-26T10:57:52-07:00', 'modified_at': '2025-08-25T13:51:12-07:00', 'language': 'en', 'timezone': 'America/Los_Angeles', 'space_amount': 999999999999999, 'space_used': 3609201911, 'max_upload_size': 536870912000, 'status': 'active', 'job_title': '', 'phone': '', 'address': '', 'avatar_url': 'https://hurrey.app.box.com/api/avatar/large/19498290761'}


In [16]:
from box_sdk_gen import (
    CreateAiAskMode,
    AiItemAsk,
    AiItemAskTypeField,
    AiDialogueHistory
)

prompt = """
Analyze the drug trial and create a summary approach used to test the drug.
"""

ai_response = client.ai.create_ai_ask(
    mode=CreateAiAskMode.SINGLE_ITEM_QA,
    prompt=prompt,
    items=[
        AiItemAsk(
            id=BOX_HUBS_ID, 
            type=AiItemAskTypeField.HUBS,
        )
    ]
)

print (ai_response.answer)

The Phase 2 clinical trial of Novostatin (Novitor) was designed as a randomized, double-blind, placebo-controlled, multi-center study to evaluate the safety and efficacy of Novostatin in patients with hypercholesterolemia. The study enrolled approximately 200 adult subjects aged 18–75 years with LDL cholesterol ≥130 mg/dL despite lifestyle modifications. Subjects were randomized 1:1 to receive either Novostatin 10 mg once daily or matching placebo for 24 weeks. Key inclusion criteria included BMI between 18 and 35 kg/m², with protocol amendments allowing dose escalation to 15 mg at Week 8 for patients with BMI >30 kg/m² who showed insufficient LDL reduction and no safety concerns.

Primary efficacy assessment focused on the mean percentage change in LDL cholesterol from baseline to Week 24, analyzed using an ANCOVA model adjusting for baseline LDL levels. Secondary endpoints included changes in total cholesterol, HDL cholesterol, triglycerides, inflammatory biomarkers (hs-CRP, IL-6), a

In [17]:
dialogue_history = []

dialogue_history.append(AiDialogueHistory(
    prompt=prompt,
    answer=ai_response.answer, # type: ignore
    created_at=ai_response.created_at # type: ignore
))

new_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    "Did the drug work better than a placebo?",
    [
        AiItemAsk(
            id=BOX_HUBS_ID,
            type=AiItemAskTypeField.HUBS,
        )
    ],
    dialogue_history=dialogue_history,
    include_citations=True
)

print (new_response.answer)
print (f"Citations: {new_response.citations}")

Yes, Novostatin demonstrated significantly better efficacy than placebo in the Phase 2 trial. The primary endpoint showed a mean LDL cholesterol reduction of 20.5% from baseline to Week 24 in the Novostatin group compared to only 4.8% in the placebo group, with a highly statistically significant between-group difference (p < 0.001). Secondary lipid parameters also improved significantly with Novostatin versus placebo: total cholesterol decreased by 15.0% vs. 3.5%, HDL cholesterol increased by 5.0% vs. 1.0%, and triglycerides decreased by 12.0% vs. 2.0% (all p-values < 0.01). Additionally, inflammatory biomarkers such as hs-CRP were reduced by approximately 25% with Novostatin compared to 5% with placebo (p < 0.001), and carotid intima-media thickness showed a significant decrease, indicating potential vascular benefits.

These results collectively confirm that Novostatin was more effective than placebo in lowering LDL cholesterol and improving cardiovascular risk markers.

Would you li

In [ ]:
%%writefile box_ai_qna_hub.py
import asyncio
import os

from dotenv import load_dotenv

from box_sdk_gen import (
    AiDialogueHistory,
    AiItemAsk,
    AiItemAskTypeField,
    BoxClient,
    BoxCCGAuth,
    CCGConfig,
    CreateAiAskMode
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_HUBS_ID=os.getenv('HUBS_ID')

async def chat_with_ai(): 
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )
    auth = BoxCCGAuth(config=ccg_config)
    client = BoxClient(auth=auth)
    
    dialogue_history = []
    
    print("Box AI Chat - Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        prompt = input("\nYou: ").strip()
        
        if prompt.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
            
        if not prompt:
            continue
            
        try:
            response = client.ai.create_ai_ask(
                mode=CreateAiAskMode.SINGLE_ITEM_QA,
                prompt=prompt,
                items=[
                    AiItemAsk(
                        id=BOX_HUBS_ID, 
                        type=AiItemAskTypeField.HUBS,
                    )
                ],
                dialogue_history=dialogue_history,
                include_citations=True,
            )
            
            print(f"\nAI: {response.answer}") # type: ignore
            
            i=1
            print("\nCitations:")
            print("-" * 20)
            for citation in response.citations:
                print(f"\n{i}. {citation.content} ({citation.name})") # type: ignore
                i += 1
            
            # Update dialogue history
            dialogue_history.append(AiDialogueHistory(
                prompt=prompt,
                answer=response.answer, # type: ignore
                created_at=response.created_at # type: ignore
            ))
            
        except Exception as e:
            print(f"\nError: {e}")

if __name__ == "__main__":
    asyncio.run(chat_with_ai())


Overwriting box_ai_qna_hub.py
